In [1]:
from keras.models import load_model
from collections import deque   
import numpy as np
import pickle
import cv2
from PIL import Image as im 
import json
from urllib.request import urlopen
import firebase_admin
from firebase_admin import credentials,db,storage,messaging

In [2]:
# idhr appne system ke according jo mai model bhejunga uska path and pickle ka bhi
# model = load_model(r'C:\Users\hp\Downloads\data\video_model\model') 
model = load_model(r"C:\Users\hp\Desktop\final models rj police\violence_model\model\model.h5") 
# lb = pickle.loads(open(r"C:\Users\hp\Downloads\data\video_model\labelbinarizer.pickle", "rb").read())
# lb = pickle.loads(open(r"C:\Users\hp\Desktop\aarohi_model\label_binarizer.pickle", "rb").read())
# print(pickle.loads(open(r"C:\Users\hp\Downloads\data\video_model\labelbinarizer.pickle", "rb").read()))
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

smtp_port = 587
smtp_server = "smtp.gmail.com"
cred = credentials.Certificate("credentials.json")
firebase_admin.initialize_app(cred,{"storageBucket":"rajasthanpolice-bc516.appspot.com"})
bucket = storage.bucket()
registration_token = 'ctSdcSSDSZOwPqBYoy-nBJ:APA91bHEpljSq8xxAJiTtD2IvTQS9fjhb8OvoCpFNCK6n7ljiEhpbxQRUvHNoKeK5MEgvPHWICE7iyvk8it4Uu2joxu01uT4HwyFBbkBqXqCgkEfciJ2o_kSm2FkBUMVfU8af3YOz3OE'


In [3]:
# anomaly_label = ['Arson','Assault','Burglary','Explosion','Fighting','Normal','Road_Accident','Robbery','Shooting','Vandalism']
# anomaly_label = ['Arson','Fighting_Assault','Normal','Road_Accident','Robbery_Burglary','Shooting','Vandalism']
# anomaly_label = ['true','false']
# anomaly_label = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']
capture_video = cv2.VideoCapture(r"C:\Users\hp\Downloads\Real Street Fight Caught On CCTV Camera • Delhi Street With Criminals _ Real Fight.mp4") 
# 7pne hisab se video ka path dedena wrna 0 for webcamac
writer = None
widht,height = None,None

while True:
    taken,frame = capture_video.read()  
    if not taken:
        break
    if widht is None or height is None:
        height,widht = frame.shape[:2]  
    output = frame.copy()
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(128,128)).astype("float16")
    frame = frame.reshape(128, 128, 3) / 255
#     frame -= mean
    preds = model.predict(np.expand_dims(frame,axis=0))[0]
    print(preds)
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
#         i = np.argmax(results)
    i = (preds > 0.98)[0] #np.argmax(results)
    label = i
    print(results)
    print(i)
#     label = i
    if(label):
        data = im.fromarray(output) 
        data.save('violenceDummy.jpg') 

        url = "http://ipinfo.io/json"
        response = urlopen(url)
        data = json.load(response)
        print(data)
        message = "{data}".format(data=data)
        image =  open("violenceDummy.jpg", 'rb').read() 
        email_from = "kshitizagarwal1710@gmail.com"
        email_to = "kshitizagarwal1710@gmail.com"
        psw = "zzlpuafbnfcqbvfy"
        msg = MIMEMultipart()
        msg['Subject'] = 'VioLence Detected'
        msg['From'] = 'kshitizagarwal1710@gmail.com'
        msg['To'] = 'kshitizagarwal1710@gmail.com'
        text = MIMEText(message)
        msg.attach(text)
        image = MIMEImage(image)
        msg.attach(image)

        txt = msg.as_string()

        TIE = smtplib.SMTP(smtp_server, smtp_port) 
        TIE.starttls()
        TIE.login(email_from, psw)
        print("login success")

        print("sending mail to{email_to}")
        TIE.sendmail(email_from, email_to, txt)
        print("mail sent")
        TIE.quit()
        print("Sending image to cloud")
        blob = bucket.blob("violenceDummy.jpg")
        blob.upload_from_filename("violenceDummy.jpg")
        print("image sent to cloud")
        message = messaging.Message(
            data={
                'title': 'new Update ',
                'body': 'Violence ereupted'
            },
            token=registration_token)

# Send a message to the device corresponding to the provided
# registration token.
        response = messaging.send(message)
# Response is a message ID string.
        print('Successfully sent message:', response)
        break

    print(label)
    text = "violence: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,1.25, (255,0,0), 5)

#     if writer is None:
#         fourcc = cv2.VideoWriter_fourcc(*"MJPG")
#         writer = cv2.VideoWriter(r"C:\Users\hp\Downloads\data\video_model\output\output.avi", fourcc, 30,(widht, height), True)
#         writer.write(output)
    cv2.imshow("Output", output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print("Cleaning up...")
# writer.release()
capture_video.release()

1/1 [==============================] - 5s 5s/step
[0.82331806]
[0.82331806]
False
False
1/1 [==============================] - 0s 191ms/step
[0.9539402]
[0.88862914]
False
False
1/1 [==============================] - 0s 147ms/step
[0.88115525]
[0.8861378]
False
False
1/1 [==============================] - 0s 140ms/step
[0.8939176]
[0.88808274]
False
False
1/1 [==============================] - 0s 134ms/step
[0.88042444]
[0.8865511]
False
False
1/1 [==============================] - 0s 152ms/step
[0.9446309]
[0.8962311]
False
False
1/1 [==============================] - 0s 138ms/step
[0.8660666]
[0.8919219]
False
False
1/1 [==============================] - 0s 106ms/step
[0.8421424]
[0.8856994]
False
False
1/1 [==============================] - 0s 146ms/step
[0.85233736]
[0.8819925]
False
False
1/1 [==============================] - 0s 142ms/step
[0.8720913]
[0.8810024]
False
False
1/1 [==============================] - 0s 145ms/step
[0.8807972]
[0.88098377]
False
False
1/1 [===========